In [ ]:
"""

當用戶想要重新觀看歡迎訊息時，可輸入特定文字，Server會重新寄發歡迎訊息

當用戶發送指定文字消息時，會發送相應的圖片消息給用戶

"""

In [1]:
"""

啟用伺服器基本樣板

"""

# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)

# 載入json處理套件
import json

# 載入基礎設定檔
secretFileContentJson=json.load(open("../line_secret_key",'r'))
server_url=secretFileContentJson.get("server_url")

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/food" , static_folder = "../food/")

# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'


In [2]:
'''

製作文字與圖片發送消息

'''

# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)

# 設定消息素材
text_reply_message1 = TextSendMessage(text="歡迎光臨無人商店")
text_reply_message2 = TextSendMessage(text="更多資訊請點選下列選單")
image_reply_message1 = ImageSendMessage(
                                        original_content_url='https://%s/images/003.jpeg' %server_url ,
                                        preview_image_url='https://%s/images/001.jpg' %server_url)
image_reply_message2 = ImageSendMessage(
                                        original_content_url='https://%s/images/004.png' %server_url,
                                        preview_image_url='https://%s/images/005.jpg' %server_url)

'''

製作文字與圖片的教學訊息

'''
# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)

# 消息清單
reply_message_list = [
TextSendMessage(text="歡迎光臨無人商店"),
    TextSendMessage(text="更多資訊請點選下列選單")
]

'''

撰寫用戶關注時，我們要處理的商業邏輯

1. 取得用戶個資，並存回伺服器
2. 把先前製作好的自定義菜單，與用戶做綁定
3. 回應用戶，歡迎用的文字消息與圖片消息

'''


# 載入Follow事件
from linebot.models.events import (
    FollowEvent
)

# 載入requests套件
import requests


# 告知handler，如果收到FollowEvent，則做下面的方法處理
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    
    # 取出消息內User的資料
    user_profile = line_bot_api.get_profile(event.source.user_id)
        
     # 將用戶資訊存在檔案內
    with open("../foodusers.txt", "a") as myfile:
        myfile.write(json.dumps(vars(user_profile),sort_keys=True))
        myfile.write('\r\n')
        
        
    # 將菜單綁定在用戶身上
    linkRichMenuId=secretFileContentJson.get("rich_menu_id")
    linkMenuEndpoint='https://api.line.me/v2/bot/user/%s/richmenu/%s' % (event.source.user_id, linkRichMenuId)
    linkMenuRequestHeader={'Content-Type':'image/jpeg','Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}
    lineLinkMenuResponse=requests.post(linkMenuEndpoint,headers=linkMenuRequestHeader)
    
    # 回覆文字消息與圖片消息
    line_bot_api.reply_message(
        event.reply_token,
        reply_message_list
    )


In [3]:
'''

建立圖片消息素材

'''

# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    ImageSendMessage, TextSendMessage
)

# 圖片消息的基本建構特徵
image_message = TextSendMessage(text='2019-01-23\n雞肉(1)($ 80)\n麵粉(1)($ 50)\n------------------\nTotal  $130')
image_message2 = TextSendMessage(text='暫無紀錄')
image_message3 = ImageSendMessage(
    original_content_url='https://%s/images/preview3.png' % server_url,
    preview_image_url='https://%s/images/preview3.png' % server_url
)

image_message4 = ImageSendMessage(
    original_content_url='https://%s/images/preview4.png' % server_url,
    preview_image_url='https://%s/images/preview4.png' % server_url
)

In [4]:
'''

設計一個Carousel Flex



'''

flexCarouselContainerJsonDict = """
{
  
    "type": "carousel",
    "contents": [
      {
        "type": "bubble",
        "hero": {
          "type": "image",
          "url": "https://pgw.udn.com.tw/gw/photo.php?u=https://uc.udn.com.tw/photo/2018/05/29/draft/4804805.jpeg&x=0&y=0&sw=0&sh=0&exp=3600",
          "size": "full",
          "aspectRatio": "20:13",
          "aspectMode": "cover"
        },
        "body": {
          "type": "box",
          "layout": "vertical",
          "spacing": "sm",
          "contents": [
            {
              "type": "text",
              "text": "雞肉",
              "size": "xl",
              "weight": "bold",
              "wrap": true
            },
            {
              "type": "box",
              "layout": "baseline",
              "contents": [
                {
                  "type": "text",
                  "text": "$80 --> $70",
                  "flex": 0,
                  "size": "xl",
                  "weight": "bold",
                  "wrap": true
                }
              ]
            }
          ]
        },
        "footer": {
          "type": "box",
          "layout": "vertical",
          "spacing": "sm",
          "contents": [
            {
              "type": "button",
              "action": {
                "type": "uri",
                "label": "看食譜",
                "uri": "https://www.google.com/search?q=雞肉&rlz=1C1SQJL_zh-TWTW817TW817&source=lnms&tbm=isch&sa=X&ved=0ahUKEwjskrXvqITgAhUB9LwKHWnHBTIQ_AUIDigB&biw=1920&bih=938#imgrc=sp7sRBA8s_v8HM:"
              },
              "color": "#AAAAAA",
              "style": "primary"
            }
          ]
        }
      },
      {
        "type": "bubble",
        "hero": {
          "type": "image",
          "url": "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQgszxK4wWxfhfCokY5PwlL52bzi6FTa7vILrrewvOPW6-jDeYGzg",
          "size": "full",
          "aspectRatio": "20:13",
          "aspectMode": "cover"
        },
        "body": {
          "type": "box",
          "layout": "vertical",
          "spacing": "sm",
          "contents": [
            {
              "type": "text",
              "text": "麵粉",
              "size": "xl",
              "weight": "bold",
              "wrap": true
            },
            {
              "type": "box",
              "layout": "baseline",
              "flex": 1,
              "contents": [
                {
                  "type": "text",
                  "text": "$50 -> $35",
                  "flex": 0,
                  "size": "xl",
                  "weight": "bold",
                  "wrap": true
                }
              ]
            },
            {
              "type": "text",
              "text": "Temporarily out of stock",
              "flex": 0,
              "margin": "md",
              "size": "xxs",
              "color": "#FF5551",
              "wrap": true
            }
          ]
        },
        "footer": {
          "type": "box",
          "layout": "vertical",
          "spacing": "sm",
          "contents": [
            {
              "type": "button",
              "action": {
                "type": "uri",
                "label": "看食譜",
                "uri": "https://www.google.com/search?rlz=1C1SQJL_zh-TWTW817TW817&biw=1920&bih=938&tbm=isch&sa=1&ei=BWhIXIugCcLQ8wXanZCwCw&q=麵粉&oq=麵粉&gs_l=img.3.5.0l10.128059.129167..130167...0.0..0.49.281.7....2..1....1..gws-wiz-img.OpZeLcsdNEI#imgrc=LnVwhL_CkyFzEM:"
              },
              "flex": 2,
              "color": "#AAAAAA",
              "style": "primary"
            }
          ]
        }
      },
      {
        "type": "bubble",
        "hero": {
          "type": "image",
          "url": "https://www.foodnext.net/dispPageBox/getFile/GetImg.aspx?FileLocation=/PJ-FOODNEXT/Files/&FileName=photo-08083-i.jpg",
          "size": "full",
          "aspectRatio": "20:13",
          "aspectMode": "cover"
        },
        "body": {
          "type": "box",
          "layout": "vertical",
          "spacing": "sm",
          "contents": [
            {
              "type": "text",
              "text": "醬油",
              "size": "xl",
              "weight": "bold",
              "wrap": true
            },
            {
              "type": "box",
              "layout": "baseline",
              "contents": [
                {
                  "type": "text",
                  "text": "$70 -> $60",
                  "flex": 0,
                  "size": "xl",
                  "weight": "bold",
                  "wrap": true
                }
              ]
            }
          ]
        },
        "footer": {
          "type": "box",
          "layout": "vertical",
          "spacing": "sm",
          "contents": [
            {
              "type": "button",
              "action": {
                "type": "uri",
                "label": "看食譜",
                "uri": "https://www.google.com/search?q=醬油&rlz=1C1SQJL_zh-TWTW817TW817&source=lnms&tbm=isch&sa=X&ved=0ahUKEwjkj7qcqoTgAhXGWbwKHXDACkcQ_AUIDigB&biw=1920&bih=938#imgrc=unBfcGa1SM7KeM:"
              },
              "color": "#AAAAAA",
              "style": "primary"
            }
          ]
        }
      },
      {
        "type": "bubble",
        "hero": {
          "type": "image",
          "url": "https://cw1.tw/CW/images/article/201705/article-59099567d47cb.jpg",
          "size": "full",
          "aspectRatio": "20:13",
          "aspectMode": "cover"
        },
        "body": {
          "type": "box",
          "layout": "vertical",
          "spacing": "sm",
          "contents": [
            {
              "type": "text",
              "text": "雞蛋",
              "size": "xl",
              "weight": "bold",
              "wrap": true
            },
            {
              "type": "box",
              "layout": "baseline",
              "flex": 1,
              "contents": [
                {
                  "type": "text",
                  "text": "$40 -> $30",
                  "flex": 0,
                  "size": "xl",
                  "weight": "bold",
                  "wrap": true
                }
              ]
            },
            {
              "type": "text",
              "text": "Temporarily out of stock",
              "flex": 0,
              "margin": "md",
              "size": "xxs",
              "color": "#FF5551",
              "wrap": true
            }
          ]
        },
        "footer": {
          "type": "box",
          "layout": "vertical",
          "spacing": "sm",
          "contents": [
            {
              "type": "button",
              "action": {
                "type": "uri",
                "label": "看食譜",
                "uri": "https://www.google.com/search?rlz=1C1SQJL_zh-TWTW817TW817&biw=1920&bih=938&tbm=isch&sa=1&ei=1GhIXMmrL4O58QWVtYbAAg&q=雞蛋&oq=雞蛋&gs_l=img.3..0l10.73974.76167..76797...0.0..0.51.405.10....2..1....1..gws-wiz-img.....0.XaiHMfPYzrg#imgrc=8tEC1TpVmtZRhM:"
              },
              "flex": 2,
              "color": "#AAAAAA",
              "style": "primary"
            }
          ]
        }
      },
      {
        "type": "bubble",
        "body": {
          "type": "box",
          "layout": "vertical",
          "spacing": "sm",
          "contents": [
            {
              "type": "button",
              "action": {
                "type": "uri",
                "label": "See more",
                "uri": "https://en.oxforddictionaries.com/definition/cuisine"
              },
              "flex": 1,
              "gravity": "center"
            }
          ]
        }
      }
    ]
  
}

"""

In [5]:
'''

將carousel類型的json 進行轉換變成 Python可理解之類型物件

將該物件封裝進 Flex Message中


'''

from linebot.models import(
    FlexSendMessage,CarouselContainer
)

import json

carouselContent = CarouselContainer.new_from_json_dict(json.loads(flexCarouselContainerJsonDict))
flexCarouselSendMeesage =  FlexSendMessage(alt_text="hello", contents=carouselContent)

In [6]:
'''

設計一個字典
    當用戶輸入相應文字消息時，系統會從此挑揀消息

'''

# 根據自定義菜單四張故事線的圖，設定相對應image
template_message_dict = {
    "[::text:]本次交易":image_message,
    "[::text:]歷史交易":image_message2,
    "[::flex:]促銷食材":flexCarouselSendMeesage,
    #"[::text:]多組織多位公證人":image_message4,
    #"[::text:]教學訊息":[text_reply_message1,text_reply_message2,image_reply_message1,image_reply_message2]
}

In [7]:
'''

當用戶發出文字消息時，判斷文字內容是否包含[::text:]，
    若有，則從template_message_dict 內找出相關訊息
    若無，則回傳預設訊息。

'''

# 用戶發出文字消息時， 按條件內容, 回傳文字消息
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    
    if(event.message.text.find('::text:')!= -1) or (event.message.text.find('::flex:')!= -1):
        line_bot_api.reply_message(
        event.reply_token,
        template_message_dict.get(event.message.text)
        )
    else:
        line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text="請點擊菜單上圖面，或輸入[::text:]more，取得更多幫助")
        )

In [ ]:
'''

執行此句，啟動Server，觀察後，按左上方塊，停用Server

'''

if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
172.17.0.3 - - [24/Jan/2019 09:05:18] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [24/Jan/2019 09:05:20] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [24/Jan/2019 09:05:22] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [24/Jan/2019 09:54:49] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [24/Jan/2019 09:54:51] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [24/Jan/2019 09:54:57] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [24/Jan/2019 09:55:01] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [24/Jan/2019 09:55:09] "POST / HTTP/1.1" 200 -
